## Python で始める機器分析データの解析とケモメトリックス  

### 11.1. 一般化二次元相関分光法

In [ ]:
import numpy
import pandas
from matplotlib import pyplot

In [ ]:
filename = "data7.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = pandas.read_csv(filename, header=None, index_col=0).iloc[0].values
data.T.plot()

ダイナミックスペクトル  
$$ \tilde{y}(\nu, t)=y(\nu, t)-\bar{y}(\nu) $$

$$ \tilde{y}(\nu, t) = \left\{ \begin{array}{c} y(\nu, t)-\bar{y}(\nu) & \textrm{for   } T_{\textrm{min}} \le t \le T_{\textrm{max}} \\ 0 & \textrm{otherwise}
\end{array}
\right. $$

In [ ]:
data -= data.mean()
data.T.plot(legend = None)

In [ ]:
def contour(data):
    x = data.columns[0:].astype(float)
    y = data.index[0:].astype(float)
    z = data.values
    zmax = numpy.absolute(z).max()
    pyplot.figure(figsize=(6, 6))
    num_contour = 16
    pyplot.contour(x, y, z, num_contour, cmap="bwr", vmin=-1 * zmax, vmax=zmax)

同時相関スペクトル  
$$\Phi (\nu_1, \nu_2)=\frac{1}{n-1} \sum_{j=1}^{n} \tilde{y}(\nu_1, t_j) \cdot \tilde{y}(\nu_2, t_j) $$

In [ ]:
sync = pandas.DataFrame(data.T.values @ data.values / (len(data) - 1), index=data.columns, columns=data.columns).T
contour(sync)

In [ ]:
contour(data.cov(ddof=1))

ヒルベルト-野田変換行列  
$$ M_{jk} = \left\{ \begin{array}{c} 0 & \textrm{if   } j=k \\ \displaystyle \frac{1}{\pi(k-j)} & \textrm{otherwise}
\end{array}
\right. $$

In [ ]:
from math import pi
noda = numpy.zeros([len(data), len(data)])
for i, j in numpy.ndindex(noda.shape):
	if i != j: noda[i, j] = 1 / pi / (j - i)

異時相関スペクトル  
$$\Psi (\nu_1, \nu_2)=\frac{1}{n-1} \sum_{j=1}^{n} \tilde{y}(\nu_1, t_j) \cdot \sum_{k=1}^{n} M_{jk} \cdot  \tilde{y}(\nu_2, t_k) $$

In [ ]:
asyn = pandas.DataFrame(data.T.values @ noda @ data.values / (len(data) - 1), index=data.columns, columns=data.columns).T
contour(asyn)

(c) Shigeaki Morita